# Introduction
In this project I trained a classifier to distinguish between the two genres (Hip-Hop and Rock) based only on track information derived from Echonest (now part of Spotify). I first made use of pandas and seaborn packages in Python for subsetting the data, aggregating information, and creating plots when exploring the data for obvious trends or factors you should be aware of when doing machine learning.
Next, I used the scikit-learn package to predict whether I can correctly classify a song's genre based on features such as danceability, energy, acousticness, tempo, etc. I went over implementations of common algorithms such as PCA, logistic regression, decision trees, and so forth.

# Preparing the data
These recommendations are so on point! How does this playlist know me so well?

Over the past few years, streaming services with huge catalogs have become the primary means through which most people listen to their favorite music. But at the same time, the sheer amount of music on offer can mean users might be a bit overwhelmed when trying to look for newer music that suits their tastes.

For this reason, streaming services have looked into means of categorizing music to allow for personalized recommendations. One method involves direct analysis of the raw audio information in a given song, scoring the raw data on a variety of metrics. Today, we'll be examining data compiled by a research group known as The Echo Nest. Our goal is to look through this dataset and classify songs as being either 'Hip-Hop' or 'Rock' - all without listening to a single one ourselves. In doing so, we will learn how to clean our data, do some exploratory data visualization, and use feature reduction towards the goal of feeding our data through some simple machine learning algorithms, such as decision trees and logistic regression.

To begin with, let's load the metadata about our tracks alongside the track metrics compiled by The Echo Nest. A song is about more than its title, artist, and number of listens. We have another dataset that has musical features of each track such as danceability and acousticness on a scale from -1 to 1. These exist in two different files, which are in different formats - CSV and JSON. While CSV is a popular file format for denoting tabular data, JSON is another common file format in which databases often return the results of a given query.

Let's start by creating two pandas DataFrames out of these files that we can merge so we have features and labels (often also referred to as X and y) for the classification later on.



# Import Required Library

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [ ]:
rhop=pd.read_csv('../input/eechonest/fma-rock-vs-hiphop.csv')
DF=pd.read_json('../input/eechonest/echonest-metrics.json')
DF=DF.merge(rhop[['track_id','genre_top']],on='track_id')
DF.info()

# Pairwise Relationships between continous variables

We typically want to avoid using variables that have strong correlations with each other -- hence avoiding feature redundancy -- for a few reasons:

* To keep the model simple and improve interpretability (with many features, we run the risk of overfitting).
* When our datasets are very large, using fewer features can drastically speed up our computation time.
To get a sense of whether there are any strongly correlated features in our data, we will use built-in functions in the pandas package

In [ ]:
DFcor=DF.corr()
DFcor.style.background_gradient()

# Splitting our Data

As mentioned earlier, it can be particularly useful to simplify our models and use as few features as necessary to achieve the best result. Since we didn't find any particularly strong correlations between our features, we can now split our data into an array containing our features, and another containing the labels - the genre of the track.

Once we have split the data into these arrays, we will perform some preprocessing steps to optimize our model development.

In [ ]:
from sklearn.model_selection import train_test_split
y=DF['genre_top'].values
X=DF.drop(['track_id','genre_top'],axis=1).values
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=10)


# Normalising the feature 

As mentioned earlier, it can be particularly useful to simplify our models and use as few features as necessary to achieve the best result. Since we didn't find any particular strong correlations between our features, we can instead use a common approach to reduce the number of features called principal component analysis (PCA).

It is possible that the variance between genres can be explained by just a few features in the dataset. PCA rotates the data along the axis of highest variance, thus allowing us to determine the relative contribution of each feature of our data towards the variance between classes.

However, since PCA uses the absolute variance of a feature to rotate the data, a feature with a broader range of values will overpower and bias the algorithm relative to the other features. To avoid this, we must first normalize our train and test features. There are a few methods to do this, but a common way is through standardization, such that all features have a mean = 0 and standard deviation = 1 (the resultant is a z-score).

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
scaled_train=sc.fit_transform(X_train)
scaled_test=sc.transform(X_test)


# Principle Component Analysis on our data

Now that we have preprocessed our data, we are ready to use PCA to determine by how much we can reduce the dimensionality of our data. We can use scree-plots and cumulative explained ratio plots to find the number of components to use in further analyses.

Scree-plots display the number of components against the variance explained by each component, sorted in descending order of variance. Scree-plots help us get a better sense of which components explain a sufficient amount of variance in our data. When using scree plots, an 'elbow' (a steep drop from one data point to the next) in the plot is typically used to decide on an appropriate cutoff.

In [ ]:
from sklearn.decomposition import PCA
pca=PCA()
pca.fit(scaled_train)
exp_variance=pca.explained_variance_ratio_

fig,ax=plt.subplots()
ax.bar(range(pca.n_components_),exp_variance)
ax.set(xlabel='Principle Components')

# Furthur visualization of PCA
Unfortunately, there does not appear to be a clear elbow in this scree plot, which means it is not straightforward to find the number of intrinsic dimensions using this method.

But all is not lost! Instead, we can also look at the cumulative explained variance plot to determine how many features are required to explain, say, about 85% of the variance (cutoffs are somewhat arbitrary here, and usually decided upon by 'rules of thumb'). Once we determine the appropriate number of components, we can perform PCA with that many components, ideally reducing the dimensionality of our data.

In [ ]:
fig,ax=plt.subplots()
cumvar=np.cumsum(exp_variance)
ax.plot(cumvar)
ax.axhline(y=0.85,linestyle='--')


# Projecting onto our feature
We saw from the plot that 6 features (remember indexing starts at 0) can explain 85% of the variance!

Therefore, we can use 6 components to perform PCA and reduce the dimensionality of our train and test features.

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=6,random_state=10)
pca_train=pca.fit_transform(scaled_train)
pca_test=pca.transform(scaled_test)


# Train a decision tree to train our model
Now we can use the lower dimensional PCA projection of the data to classify songs into genres.

Here, we will be using a simple algorithm known as a decision tree. Decision trees are rule-based classifiers that take in features and follow a 'tree structure' of binary decisions to ultimately classify a data point into one of two or more categories. In addition to being easy to both use and interpret, decision trees allow us to visualize the 'logic flowchart' that the model generates from the training data.

Here is an example of a decision tree that demonstrates the process by which an input image (in this case, of a shape) might be classified based on the number of sides it has and whether it is rotated.



In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier(random_state=10)
tree.fit(pca_train,y_train)
preddt=tree.predict(pca_test)
preddt

# Compare our Decision tree to a logistic Regression
Although our tree's performance is decent, it's a bad idea to immediately assume that it's therefore the perfect tool for this job -- there's always the possibility of other models that will perform even better! It's always a worthwhile idea to at least test a few other algorithms and find the one that's best for our data.

Sometimes simplest is best, and so we will start by applying logistic regression. Logistic regression makes use of what's called the logistic function to calculate the odds that a given data point belongs to a given class. Once we have both models, we can compare them on a few performance metrics, such as false positive and false negative rate (or how many points are inaccurately classified).

In [ ]:
from sklearn.linear_model import LogisticRegression
logr=LogisticRegression()
logr.fit(X_train,y_train)
predlr=logr.predict(X_test)
predlr

In [ ]:
from sklearn.metrics import classification_report
dt=classification_report(y_test,preddt)
lr=classification_report(y_test,predlr)
print(dt)



# Balance our data for greater performance
Both our models do similarly well, boasting an average precision of 87% each. However, looking at our classification report, we can see that rock songs are fairly well classified, but hip-hop songs are disproportionately misclassified as rock songs.

Why might this be the case? Well, just by looking at the number of data points we have for each class, we see that we have far more data points for the rock classification than for hip-hop, potentially skewing our model's ability to distinguish between classes. This also tells us that most of our model's accuracy is driven by its ability to classify just rock songs, which is less than ideal.

To account for this, we can weight the value of a correct classification in each class inversely to the occurrence of data points for each class. Since a correct classification for "Rock" is not more important than a correct classification for "Hip-Hop" (and vice versa), we only need to account for differences in sample size of our data points when weighting our classes here, and not relative importance of each class.

In [ ]:
hiop=DF.drop(DF[DF['genre_top']!='Hip-Hop'].index,inplace=None)
rock=DF.drop(DF[DF['genre_top']!='Rock'].index,inplace=None)

rock=rock.sample(hiop.shape[0],random_state=10)
rock_hiop=pd.concat([rock,hiop])
y=rock_hiop['genre_top'].values
X=rock_hiop.drop(['track_id','genre_top'],axis=1)

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=4)
train_pca=pca.fit_transform(sc.fit_transform(X_train))
test_pca=pca.transform(sc.transform(X_test))


# Does balancing our dataset improve model bias
We've now balanced our dataset, but in doing so, we've removed a lot of data points that might have been crucial to training our models. Let's test to see if balancing our data improves model bias towards the "Rock" classification while retaining overall classification performance.

Note that we have already reduced the size of our dataset and will go forward without applying any dimensionality reduction. In practice, we would consider dimensionality reduction more rigorously when dealing with vastly large datasets and when computation times become prohibitively large.

In [ ]:
tree=DecisionTreeClassifier(random_state=10)
tree.fit(train_pca,y_train)
predt=tree.predict(test_pca)


logr=LogisticRegression(random_state=10)
logr.fit(train_pca,y_train)
predr=logr.predict(test_pca)



print(f"Tree:{classification_report(predt,y_test)}, \n Logr: {classification_report(predr,y_test)}")

# Using cross validaton to evaluate our models
Success! Balancing our data has removed bias towards the more prevalent class. To get a good sense of how well our models are actually performing, we can apply what's called cross-validation (CV). This step allows us to compare models in a more rigorous fashion.

Before we can perform cross-validation we will need to create pipelines to scale our data, perform PCA, and instantiate our model of choice - DecisionTreeClassifier or LogisticRegression.

Since the way our data is split into train and test sets can impact model performance, CV attempts to split the data multiple ways and test the model on each of the splits. Although there are many different CV methods, all with their own advantages and disadvantages, we will use what's known as K-fold CV here. K-fold first splits the data into K different, equally sized subsets. Then, it iteratively uses each subset as a test set while using the remainder of the data as train sets. Finally, we can then aggregate the results from each fold for a final model performance score.

In [ ]:
from sklearn.model_selection import KFold,cross_val_score
from sklearn.pipeline import Pipeline
tree_pip=Pipeline([("sc",StandardScaler()),('pca',PCA(n_components=6)),('tree',DecisionTreeClassifier(random_state=10))])
logr_pip=Pipeline([("sc",StandardScaler()),('pca',PCA(n_components=6)),('logr',LogisticRegression(random_state=10))])

kf=KFold(10)
tree_score=cross_val_score(tree_pip,X,y,cv=kf)
logr_score=cross_val_score(logr_pip,X,y,cv=kf)
                           
print(np.mean(tree_score),np.mean(logr_score))

# Conclusion
The model accuracy with Decision Tree and Logistic Regression are 0.75824,
0.78296 respectively.Though accuracy both models are close enough to classify any song,Logistic Regression has a better score here.